In [1]:
import pandas as pd
import geopandas as gpd
import shapely

In [4]:
def convertGeometries(df):
    for i in range(len(df)):
        geom_type = df.loc[i, 'GeometryType']
        geom = None
        if geom_type == 'Point':
            geom =shapely.geometry.Point(df.loc[0,'Coordinates'])
        elif geom_type == 'GeometryCollection':
            if df.loc[i,'Coordinates'][0] == df.loc[i,'Coordinates'][-1]:
                geom = shapely.geometry.Polygon(df.loc[i,'Coordinates'])
            else:
                geom = shapely.geometry.LineString(df.loc[i,'Coordinates'])
        df.loc[i, 'geometry'] = geom

In [5]:
df = pd.read_csv('https://raw.githubusercontent.com/robisoniv/navwarning-analysis/master/CleanedData.csv')
df = df.drop(columns=['Unnamed: 0'])
df['StartDate'] = pd.to_datetime(df['StartDate'])
df['EndDate'] = pd.to_datetime(df['EndDate'])
df['BroadcastingDate'] = pd.to_datetime(df['BroadcastingDate'])
df['Coordinates'] = df['Coordinates'].apply(lambda c: eval(c))
convertGeometries(df)

In [26]:
gdf = gpd.GeoDataFrame(df)
gdf['centroid'] = gdf['geometry'].centroid
for i in range(len(gdf)):
    gdf.loc[i, 'lon_centroid'] = list(gdf.loc[i, 'centroid'].coords[0])[0]
    gdf.loc[i, 'lat_centroid'] = list(gdf.loc[i, 'centroid'].coords[0])[1]

In [66]:
gdf.head()

,Area,Chart,Message,Authority,Date,StartDate,EndDate,BroadcastingDate,GeometryType,geometry,Coordinates,centroid,lon_centroid,lat_centroid
0,CANADA-NORTH COAST. /BEAUFORT SEA.,15940,BOTTOM SCIENTIFIC MOORING EXTENDING 3.5 METERS...,NAVAREA XVII,No date listed,2018-06-01,2018-06-01,2018-06-01,Point,POINT Z (-127.688834 70.55800000000001 0),"[(-127.688834, 70.558, 0.0)]",POINT (-127.688834 70.55800000000001),-127.688834,70.558
1,ARCTIC. /BEAUFORT SEA.,15945,"SCIENTIFIC MOORINGS, TOP FLOAT 29 METERS, ESTA...",NAVAREA XVII,No date listed,2018-06-01,2018-06-01,2018-06-01,Point,POINT Z (-127.688834 70.55800000000001 0),"[(-133.714, 70.0585, 0.0)]",POINT (-127.688834 70.55800000000001),-127.688834,70.558
2,ARCTIC. /BEAUFORT SEA.,15945,"SCIENTIFIC MOORINGS, TOP FLOAT 29 METERS, ESTA...",NAVAREA XVII,No date listed,2018-06-01,2018-06-01,2018-06-01,Point,POINT Z (-127.688834 70.55800000000001 0),"[(-133.717167, 70.0585, 0.0)]",POINT (-127.688834 70.55800000000001),-127.688834,70.558
3,ARCTIC./CHUCKCHI SEA.,16003,"SCIENTIFIC MOORING, TOP FLOAT 971 FEET, ESTABL...",CGC HEALY,No date listed,2018-06-01,2018-06-01,2018-06-01,Point,POINT Z (-127.688834 70.55800000000001 0),"[(-158.410434, 72.7996, 0.0)]",POINT (-127.688834 70.55800000000001),-127.688834,70.558
4,ARCTIC./CHUKCHI SEA.,16003.,DNC 27. SCIENTIFIC MOORING ESTABLISHED VICINIT...,NAVAREA XVII,No date listed,2018-06-01,2018-06-01,2018-06-01,Point,POINT Z (-127.688834 70.55800000000001 0),"[(-158.702167, 72.615167, 0.0)]",POINT (-127.688834 70.55800000000001),-127.688834,70.558


In [61]:
gdf

,Area,Chart,Message,Authority,Date,StartDate,EndDate,BroadcastingDate,GeometryType,geometry,Coordinates,centroid
0,CANADA-NORTH COAST. /BEAUFORT SEA.,15940,BOTTOM SCIENTIFIC MOORING EXTENDING 3.5 METERS...,NAVAREA XVII,No date listed,2018-06-01,2018-06-01,2018-06-01,Point,POINT Z (-127.688834 70.55800000000001 0),"[(-127.688834, 70.558, 0.0)]",POINT (-127.688834 70.55800000000001)
1,ARCTIC. /BEAUFORT SEA.,15945,"SCIENTIFIC MOORINGS, TOP FLOAT 29 METERS, ESTA...",NAVAREA XVII,No date listed,2018-06-01,2018-06-01,2018-06-01,Point,POINT Z (-127.688834 70.55800000000001 0),"[(-133.714, 70.0585, 0.0)]",POINT (-127.688834 70.55800000000001)
2,ARCTIC. /BEAUFORT SEA.,15945,"SCIENTIFIC MOORINGS, TOP FLOAT 29 METERS, ESTA...",NAVAREA XVII,No date listed,2018-06-01,2018-06-01,2018-06-01,Point,POINT Z (-127.688834 70.55800000000001 0),"[(-133.717167, 70.0585, 0.0)]",POINT (-127.688834 70.55800000000001)
3,ARCTIC./CHUCKCHI SEA.,16003,"SCIENTIFIC MOORING, TOP FLOAT 971 FEET, ESTABL...",CGC HEALY,No date listed,2018-06-01,2018-06-01,2018-06-01,Point,POINT Z (-127.688834 70.55800000000001 0),"[(-158.410434, 72.7996, 0.0)]",POINT (-127.688834 70.55800000000001)
4,ARCTIC./CHUKCHI SEA.,16003.,DNC 27. SCIENTIFIC MOORING ESTABLISHED VICINIT...,NAVAREA XVII,No date listed,2018-06-01,2018-06-01,2018-06-01,Point,POINT Z (-127.688834 70.55800000000001 0),"[(-158.702167, 72.615167, 0.0)]",POINT (-127.688834 70.55800000000001)
5,ARCTIC./CHUKCHI SEA.,16003.,DNC 27. SCIENTIFIC MOORING ESTABLISHED VICINIT...,NAVAREA XVII,No date listed,2018-06-01,2018-06-01,2018-06-01,Point,POINT Z (-127.688834 70.55800000000001 0),"[(-158.412667, 72.800167, 0.0)]",POINT (-127.688834 70.55800000000001)
6,ARCTIC./BEAUFORT SEA./DNC 27.,15926.,"SCIENTIFIC MOORING, AT DEPTH 61 METERS, ESTABL...",NAVAREA XVII,2015-10-01 00:00:00,2018-06-01,2018-06-01,2018-06-01,Point,POINT Z (-127.688834 70.55800000000001 0),"[(-139.020667, 70.432334, 0.0)]",POINT (-127.688834 70.55800000000001)
7,ARCTIC./BEAUFORT SEA./DNC 27.,15926.,"SCIENTIFIC MOORINGS, AT DEPTH 183 METERS AND A...",NAVAREA XVII,2015-10-02 00:00:00,2018-06-01,2018-06-01,2018-06-01,Point,POINT Z (-127.688834 70.55800000000001 0),"[(-135.011, 70.869667, 0.0)]",POINT (-127.688834 70.55800000000001)
8,ARCTIC./BEAUFORT SEA./DNC 27.,15926.,"SCIENTIFIC MOORINGS, AT DEPTH 183 METERS AND A...",NAVAREA XVII,2015-10-02 00:00:00,2018-06-01,2018-06-01,2018-06-01,Point,POINT Z (-127.688834 70.55800000000001 0),"[(-135.019167, 70.869, 0.0)]",POINT (-127.688834 70.55800000000001)
9,ARCTIC./CHUKCHI SEA./DNC 27.,16005.,SUB-SURFACE SCIENTIFIC MOORINGS ESTABLISHED IN...,USCGC HEALY,2016-08-04 00:00:00,2018-06-01,2018-06-01,2018-06-01,Point,POINT Z (-127.688834 70.55800000000001 0),"[(-161.500184, 71.600117, 0.0)]",POINT (-127.688834 70.55800000000001)


In [33]:
list(gdf.loc[100,'geometry'].centroid.coords)

[(25.282906616435437, 35.969770971575635)]

In [34]:
# We will use Shapely's native functions for this


# def calculateCentroid(listOfTuples):
    
#     if len(listOfTuples) == 1:
#         return listOfTuples[0]
#     else:
#         lon_sum = 0
#         lat_sum = 0
#         z_sum = 0
#         l = len(listOfTuples)
#         for i in listOfTuples:
#             lon_sum += i[0]
#             lat_sum += i[1]
#             z_sum += i[2]

#         lon_centroid = lon_sum / l
#         lat_centroid = lat_sum / l
#         z_centroid = z_sum / l

#         return (lon_centroid, lat_centroid, z_centroid)

# calculateCentroid(df.loc[100,'Coordinates'])

(25.158333700000004, 36.056666799999995, 0.0)

AttributeError: 'Series' object has no attribute 'centroid'

In [ ]:
df.head()